In [ ]:
SELECT * from modified_data

In [ ]:
import pandas as pd
df = cell1.to_pandas()
df

In [ ]:
df.shape

In [ ]:
df.isnull().any()

In [ ]:
data = df
# 1. Clean whitespace
data = data.map(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:

from snowflake.snowpark import DataFrame
from snowflake.snowpark.context import get_active_session

session = get_active_session()
print(session)
session.use_database('SF_HACKATHON_DB')
session.use_schema('CRYPTO_COUGARS_SCHEMA')
#creating Snowpark Dataframe
df = session.table("Modified_data")
df.show()
#df.shape

In [ ]:
# Number of rows
row_count = df.count()

# Number of columns
column_count = len(df.schema.names)

print("shape",row_count,column_count)
# List of column names
print(df.schema.names)

# Print schema (data types and nullable info)
print(df.schema)
from snowflake.snowpark.functions import avg, min, max
df.select(
    avg("AGE_AT_FEEDBACK"),
    min("AGE_AT_FEEDBACK"),
    max("AGE_AT_FEEDBACK")
).show()



In [ ]:
SELECT * FROM MODIFIED_DATA

**TRANSLATING FEEDBACK TO ENGLISH**

ADDING A COLUMN NAMED 'T_FEEDBACK', WHICH HAS ALL FEEDBACKS TRANSLATED INTO ENGLSIH

In [ ]:
alter table modified_data add column T_Feedback String;

Translating the Patients Feedback to Englsih

In [ ]:

UPDATE modified_data
SET T_FEEDBACK = SNOWFLAKE.CORTEX.TRANSLATE(
    FEEDBACK,
    LANGUAGE_CODE,
    'en'
);

**SENTIMENT ANALYSIS**

In [ ]:
alter table modified_data add Sentiment_score float, Sentiment_Cat number(2), Labels String;

In [ ]:
UPDATE modified_data
SET
  Sentiment_score = SNOWFLAKE.CORTEX.SENTIMENT(T_Feedback),
  Sentiment_cat = WIDTH_BUCKET(SNOWFLAKE.CORTEX.SENTIMENT(T_Feedback), -1, 1, 10);


In [ ]:
select t_feedback, sentiment_score,sentiment_cat from modified_data

In [ ]:
UPDATE modified_data
SET Labels = CASE 
    WHEN Sentiment_score BETWEEN 0.70 AND 1.00 THEN 'Cured'
    WHEN Sentiment_score BETWEEN 0.2 AND 0.70 THEN 'Improved'
    WHEN Sentiment_score BETWEEN -0.04 AND 0.2 THEN 'No progress'
    WHEN Sentiment_score BETWEEN -0.5 AND -0.04 THEN 'Worsen'
    WHEN Sentiment_score BETWEEN -1.00 AND -0.5 THEN 'Adverse'
    ELSE 'Unknown'
END;


In [ ]:
select * from modified_data

**REPLYING BACK TO PATIENTS**

In [ ]:
ALTER TABLE modified_data ADD COLUMN REPLY STRING;


In [ ]:
UPDATE modified_data SET REPLY = SNOWFLAKE.CORTEX.COMPLETE(
    'mistral-large2',
    CONCAT(
      'You are a polite, empathetic healthcare provider. Read the following patient feedback and respond in the same language, using respectful and professional medical language, thanking the patient and providing brief but supportive advice: ', 
      FEEDBACK
    )
);

select * from modified_data